In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

import yaml
import optuna

from src.data.load import load_data
from src.data.prepare import prepare_data
from src.models.cv_iterator import leave_last_k
from src.data.features import feature_engineering
from src.data.utils import build_rank_input
from src.models.tuner import BayesianSearch
from src.models.evaluator import evaluation

In [2]:
# read config
with open('../config.yml', 'r') as file:
    config=yaml.load(file, Loader= yaml.SafeLoader)
del file

In [3]:
# load and prepare data
dfs = load_data(config=config['data_loader'])
dfs = prepare_data(dataframes=dfs)

In [4]:
# train-test split
df_train, df_test = leave_last_k(df=dfs['data'], config=config['optimization'])
df_train, df_valid = leave_last_k(df=df_train, config=config['optimization'])

In [5]:
user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

In [6]:
import pandas as pd
from src.models.candidate import candidate_generation

df_train_neg = candidate_generation(df_train, n=30, positive_sampling=False)
df_train_pos = candidate_generation(df_train, n=10, positive_sampling=True)
df_train = pd.concat([df_train.iloc[:,:3], df_train_neg, df_train_pos], ignore_index=True)

del df_train_neg, df_train_pos

df_train, df_valid = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train, df_valid)
    ]

In [7]:
# perform bayesian search
searcher = BayesianSearch(config['optimization']['hyper_params'], algorithm='XGBRanker')

def objective(trial) -> float:
    return searcher.fit(df_train, df_valid, trial)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2025-04-24 23:20:03,017] A new study created in memory with name: no-name-89780b85-87c6-470f-a5cc-a01c3fe7b8bd
[I 2025-04-24 23:20:04,627] Trial 0 finished with value: 0.9464535908838257 and parameters: {'learning_rate': 0.37078682433574506, 'gamma': 8.647628597752387, 'max_depth': 13, 'subsample': 0.9540068811041673, 'n_estimators': 100}. Best is trial 0 with value: 0.9464535908838257.
[I 2025-04-24 23:20:08,760] Trial 1 finished with value: 0.947606967801189 and parameters: {'learning_rate': 0.0808506200535963, 'gamma': 6.299288726027914, 'max_depth': 12, 'subsample': 0.5225518754583722, 'n_estimators': 327}. Best is trial 1 with value: 0.947606967801189.
[I 2025-04-24 23:20:10,809] Trial 2 finished with value: 0.946656103359856 and parameters: {'learning_rate': 0.24095828678445746, 'gamma': 8.664327458607922, 'max_depth': 13, 'subsample': 0.777121379390799, 'n_estimators': 135}. Best is trial 1 with value: 0.947606967801189.
[I 2025-04-24 23:20:12,449] Trial 3 finished with value

In [8]:
print("Best trial:")
print(study.best_trial.params)

Best trial:
{'learning_rate': 0.17423319195610582, 'gamma': 2.3067834016415834, 'max_depth': 4, 'subsample': 0.7242883370125134, 'n_estimators': 376}


In [9]:
import pandas as pd

# use indicator to know the origin
df_train = dfs['data'].merge(
    df_test
    , on=['user_id', 'item_id', 'rating'], how='left'
    , indicator=True
    )
# keep only rows that are present in df1 but not in df2
df_train = df_train[df_train['_merge'] == 'left_only'].drop(columns=['_merge'])

user_item_features = feature_engineering(
    dataframes={'user': dfs['user'], 'item': dfs['item'], 'data': df_train}
    )

# add negative sampling
df_train_neg = candidate_generation(df_train, n=20, positive_sampling=False)
df_train_pos = candidate_generation(df_train, n=20, positive_sampling=False)
df_train_ = pd.concat([df_train, df_train_neg, df_train_pos], ignore_index=True)

del df_train_neg, df_train_pos

df_train_, df_test_ = [
    build_rank_input(ratings=df.iloc[:,:3], features=user_item_features) for df in (df_train_, df_test)
    ]

In [10]:
from sklearn.metrics import ndcg_score
from xgboost import XGBRanker
import numpy as np

clf = XGBRanker(**study.best_trial.params)
clf.fit(
    df_train_['X'], df_train_['y'].astype(int)
    , group=df_train_['group']
    , verbose=False
    )

preds = clf.predict(df_test_['X'])

print(evaluation(df_test_['y'], preds, df_test_['group']))

0.9531686718041839


In [11]:
def calculate_metrics(top_items, data):
    # Coverage: proportion of unique items recommended
    recommended_items = top_items['item_id'].unique()
    total_items = data['item_id'].unique()
    coverage_score = len(recommended_items) / len(total_items)
    
    # Novelty: Based on item popularity (inverse)
    item_popularity = data.groupby('item_id').size()
    item_popularity = item_popularity[item_popularity > 0]
    item_popularity = 1 - item_popularity / item_popularity.sum()  # Normalize
    novelty_score = top_items['item_id'].map(item_popularity).mean()

    return coverage_score, novelty_score

# Calculate metrics for the recommendations
coverage_score, novelty_score = calculate_metrics(df_test.iloc[:, :2], df_train.iloc[:, :3])
# Print out the results
print(f'Coverage: {coverage_score}')
print(f'Novelty: {novelty_score}')

Coverage: 0.6463195691202872
Novelty: 0.9984584266355963


In [12]:
from src.models.baseline import baseline_model

result = baseline_model(dataframes=dfs, n=10)
group = result.groupby('user_id').size().to_list()

print(evaluation(result['rating'], result['est_rating'], group))
del group

0.9168812846621082


In [13]:
user_item_features = feature_engineering(dataframes={
    'user': dfs['user'], 'item': dfs['item'], 'data': dfs['data']
    })

df_baseline = build_rank_input(ratings=result.drop(columns=['est_rating']), features=user_item_features)

preds = clf.predict(df_baseline['X'])
print(evaluation(df_baseline['y'], preds, df_baseline['group']))

0.9494141491923237
